In [16]:
from bs4 import BeautifulSoup
import re
import urllib3
import certifi
import pandas as pd
from urllib.parse import quote
from selenium import webdriver
import json
from pandas.io.json import json_normalize

from datetime import datetime
import requests

from requests.packages.urllib3.exceptions import InsecureRequestWarning
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

In [12]:
#se define la función word_concat_search cuyo objetivo es concatenar las palabras que se van a incluir en la búsqueda en su propio formato. Por ejemplo:
# se buscan las palabras 'tra' y 'mtc'. Como resultado se obtiene'tra%20mtc'
def word_concat_search(word_search):
    return pd.DataFrame({'word':word_search}).apply('%20'.join)['word']

def number_web_search(html):
    len_page = len(html.find('table').find('ul',attrs={'class':'pagination'}).find_all('li'))
    return len_page-2

def conexion_inicial(word_search,pag=1):
    busqueda = word_concat_search(word_search)
    url = "https://www.tramites.gob.pe/buscador/psce_tramites_buscador.aspx?txt_busqueda_general="+busqueda+"&pag="+str(pag)
    r = requests.get(url , verify=False)
    if (r.status_code==200):
        print('Conectado')
        html = BeautifulSoup(r.text, "html.parser")
        return html
    else: print('no se conectó')

In [13]:
word_search = ['tra','mtc']
html=conexion_inicial(word_search)

nw = number_web_search(html)
print('Se identificaron ',nw,' páginas para scraping')

Conectado
Se identificaron  274  páginas para scraping


In [19]:
number_search = [] ; id_link = [] ; words = [] ;time=[] ; link = [] ; tramite=[] ; ministerio=[]
for j in range(nw):
    print(j,word_search)
    print(str(datetime.now()))
    html=conexion_inicial(word_search,pag=j+1)
    n_tr=len(html.find('table').find_all('tr'))
    listado_tramites = html.find('table').find_all('tr')[1:n_tr-1]

    for i in range(len(listado_tramites)):
        number_search.append(j)
        id_link.append(i)
        words.append(word_concat_search(word_search))
        time.append(str(datetime.now()))
        
        try: link.append('https://www.tramites.gob.pe/buscador/'+listado_tramites[i].find('a')['href'])
        except: link.append('Sin página')

        try: tramite.append(listado_tramites[i].find('a').text)
        except: tramite.append('Sin trámite')

        try: ministerio.append(listado_tramites[i].find_all('td')[1].text)
        except: ministerio.append('sin ministerio')

df_tramites_link=pd.DataFrame({'id_search':number_search,'word_key':words,'id_link':id_link,'link':link, 
                               'tramite':tramite,'ministerio':ministerio,'time_update':time})

0 ['tra', 'mtc']
2019-09-14 12:28:06.844718
Conectado
1 ['tra', 'mtc']
2019-09-14 12:28:09.906378
Conectado
2 ['tra', 'mtc']
2019-09-14 12:28:14.023023
Conectado
3 ['tra', 'mtc']
2019-09-14 12:28:18.451886
Conectado
4 ['tra', 'mtc']
2019-09-14 12:28:21.749491
Conectado
5 ['tra', 'mtc']
2019-09-14 12:28:23.941251
Conectado
6 ['tra', 'mtc']
2019-09-14 12:28:27.477566
Conectado
7 ['tra', 'mtc']
2019-09-14 12:28:30.281081
Conectado
8 ['tra', 'mtc']
2019-09-14 12:28:32.606941
Conectado
9 ['tra', 'mtc']
2019-09-14 12:28:36.349654
Conectado
10 ['tra', 'mtc']
2019-09-14 12:28:39.332441
Conectado
11 ['tra', 'mtc']
2019-09-14 12:28:42.010601
Conectado
12 ['tra', 'mtc']
2019-09-14 12:28:45.059900
Conectado
13 ['tra', 'mtc']
2019-09-14 12:28:47.866342
Conectado
14 ['tra', 'mtc']
2019-09-14 12:28:51.480055
Conectado
15 ['tra', 'mtc']
2019-09-14 12:28:55.852642
Conectado
16 ['tra', 'mtc']
2019-09-14 12:29:00.010781
Conectado
17 ['tra', 'mtc']
2019-09-14 12:29:04.635413
Conectado
18 ['tra', 'mtc']
20

Conectado
149 ['tra', 'mtc']
2019-09-14 12:34:49.936651
Conectado
150 ['tra', 'mtc']
2019-09-14 12:34:52.106812
Conectado
151 ['tra', 'mtc']
2019-09-14 12:34:54.271492
Conectado
152 ['tra', 'mtc']
2019-09-14 12:34:56.408791
Conectado
153 ['tra', 'mtc']
2019-09-14 12:34:58.599253
Conectado
154 ['tra', 'mtc']
2019-09-14 12:35:00.764489
Conectado
155 ['tra', 'mtc']
2019-09-14 12:35:02.919576
Conectado
156 ['tra', 'mtc']
2019-09-14 12:35:05.269258
Conectado
157 ['tra', 'mtc']
2019-09-14 12:35:07.443925
Conectado
158 ['tra', 'mtc']
2019-09-14 12:35:09.789448
Conectado
159 ['tra', 'mtc']
2019-09-14 12:35:11.976018
Conectado
160 ['tra', 'mtc']
2019-09-14 12:35:14.365759
Conectado
161 ['tra', 'mtc']
2019-09-14 12:35:17.048151
Conectado
162 ['tra', 'mtc']
2019-09-14 12:35:19.596615
Conectado
163 ['tra', 'mtc']
2019-09-14 12:35:21.767556
Conectado
164 ['tra', 'mtc']
2019-09-14 12:35:24.177022
Conectado
165 ['tra', 'mtc']
2019-09-14 12:35:27.417835
Conectado
166 ['tra', 'mtc']
2019-09-14 12:35:29

In [74]:
df_tramites=df_tramites_link[df_tramites_link.link!='Sin página'].reset_index().drop(columns=['index'])

In [102]:
ministerios = [] ; id_entidad = [] ; id_registro = [] ; tipo_registro = [] ; tipo_acceso = [];
for i in range(len(df_tramites)):
    ltra=len(str(df_tramites.tramite[0]))
    ministerios.append(str(df_tramites.ministerio[0])[ltra:])
    
    id_pagina = str(df_tramites.link[i])[58:]
    id_entidad.append(re.search(r'([0-9]*[0-9])',re.search(r'(id_entidad.*)', id_pagina).group(0)).group(0)) # id entidad
    id_registro.append(re.search(r'([0-9]*[0-9])',re.search(r'(id_registro.*)', id_pagina).group(0)).group(0)) # id registro
    tipo_registro.append(re.search(r'([A-Z])',re.search(r'(=.*&)',re.search(r'(tipo_registro.*)', id_pagina).group(0)).group(0)).group(0)) # tipo registro
    tipo_acceso.append(re.search(r'([A-Z])',re.search(r'(tipo_acceso.*)', id_pagina).group(0)).group(0)) # tipo acceso
identidad_registro = pd.DataFrame({'entidad':ministerios,'id_entidad':id_entidad,'id_registro':id_registro,'tipo_registro':tipo_registro,'tipo_acceso':tipo_acceso})
df_tramites_final = pd.concat([identidad_registro,df_tramites],axis=1)

In [104]:
df_tramites_final.columns

Index(['entidad', 'id_entidad', 'id_registro', 'tipo_registro', 'tipo_acceso',
       'id_search', 'word_key', 'id_link', 'link', 'tramite', 'ministerio',
       'time_update'],
      dtype='object')

In [106]:
df_tramites_final.to_csv('DataSet/df_tramites_link.csv',sep='|',encoding='utf8',index_label=False,index=False)

# Data de cada trámite

In [108]:
df_tramites_final=pd.read_csv('DataSet/df_tramites_link.csv',sep='|',encoding='utf8')

In [112]:
df_tramites_final.link[40]

'https://www.tramites.gob.pe/buscador/psce_reg_accion.aspx?id_entidad=181&id_registro=1020&tipo_registro=F&tipo_acceso=B'